In [1]:
#Uploading packages we'll need later

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
pd.options.display.max_columns = None 

In [2]:
#Read in raw migration data
migration_data = pd.read_csv('Raw Data/Persons of Concern/for_map_unhcr_popstats_export_persons_of_concern_all_data.csv', dtype=str)

#Rename data
migration_data = migration_data.rename(columns={'Country / territory of asylum/residence': 'Country', 'Refugees (incl. refugee-like situations)': 'Refugees', 'Asylum-seekers (pending cases)': 'Asylum', 'Internally displaced persons (IDPs)': 'Internally displaced persons'})

In [3]:
#Check it out
migration_data.head()

,Year,Country,Origin,Refugees,Asylum,Returned refugees,Internally displaced persons,Returned IDPs,Stateless persons,Others of concern,Total Population
0,1951,Australia,Various/Unknown,180000,NaN,NaN,NaN,NaN,NaN,NaN,180000
1,1951,Austria,Various/Unknown,282000,NaN,NaN,NaN,NaN,NaN,NaN,282000
2,1951,Belgium,Various/Unknown,55000,NaN,NaN,NaN,NaN,NaN,NaN,55000
3,1951,Canada,Various/Unknown,168511,NaN,NaN,NaN,NaN,NaN,NaN,168511
4,1951,Switzerland,Various/Unknown,10000,NaN,NaN,NaN,NaN,NaN,NaN,10000


In [4]:
#Make a new dataset just for refugees going into Brazil and reset the index of that dataset so it can be a dataframe
refugees_in = migration_data.loc[migration_data['Country'] == 'Brazil']
refugees_in = refugees_in.reset_index(drop=True)

#Note - there are a few values that have an * which denotes an unclear count in that category. 
#In our data, we will replace them with zeros

refugees_in.replace(['*'], [0], inplace=True) 
refugees_in.fillna(0, inplace=True)

#Convert columns to numeric where necessary. 
refugees_in[['Year','Refugees','Asylum','Returned refugees','Internally displaced persons','Returned IDPs','Stateless persons','Others of concern','Total Population']] = refugees_in[['Year','Refugees','Asylum','Returned refugees','Internally displaced persons','Returned IDPs','Stateless persons','Others of concern','Total Population']].apply(pd.to_numeric)

#Export refugees_in to CSV
refugees_in.to_csv('Processed Data/refugees_in.csv')

#Make a new dataset that summarizes in-migration of refugees by year
refugees_in_by_year = refugees_in.groupby(['Year'])['Refugees','Asylum','Returned refugees','Internally displaced persons','Returned IDPs','Stateless persons','Others of concern','Total Population'].sum()
refugees_in_by_year.head()

#Export refugees_out_by_year to CSV
refugees_in_by_year.to_csv('Processed Data/refugees_in_by_year.csv')

In [5]:
#Make a new dataset just for refugees coming out Brazil and reset the index of that dataset
refugees_out = migration_data.loc[migration_data['Origin'] == 'Brazil']
refugees_out = refugees_out.reset_index(drop=True)

#Note - there are a few values that have an * which denotes an unclear count in that category. 
#In our data, we will replace them with zeros

refugees_out.replace(['*'], [0], inplace=True) 
refugees_out.fillna(0, inplace=True)

#Convert columns to numeric where necessary. 
refugees_out[['Year','Refugees','Asylum','Returned refugees','Internally displaced persons','Returned IDPs','Stateless persons','Others of concern','Total Population']] = refugees_out[['Year','Refugees','Asylum','Returned refugees','Internally displaced persons','Returned IDPs','Stateless persons','Others of concern','Total Population']].apply(pd.to_numeric)

#Export refugees_out to CSV
refugees_out.to_csv('Processed Data/refugees_out.csv')

#Make a new dataset that summarizes out-migration of refugees by year
refugees_out_by_year = refugees_out.groupby(['Year'])['Refugees','Asylum','Returned refugees','Internally displaced persons','Returned IDPs','Stateless persons','Others of concern','Total Population'].sum()
refugees_out_by_year.head()

#Export refugees_out_by_year to CSV
refugees_out_by_year.to_csv('Processed Data/refugees_out_by_year.csv')

In [6]:
#Now merge to complete dataset to show net change over time.

refugees_net_by_year = pd.merge(refugees_in_by_year.add_suffix('_in'), refugees_out_by_year.add_suffix('_out'), left_index=True, right_index=True, how='outer')

#Clean to get rid of the NaN - fill them in with zeros
refugees_net_by_year.fillna(0, inplace=True)
refugees_net_by_year

#Write a function to get net values
out_vars = list(refugees_out_by_year.add_suffix('_out').columns.values)
in_vars = list(refugees_in_by_year.add_suffix('_in').columns.values)
root_vars = list(refugees_in_by_year.columns.values)

#Find net change over time in a given year
for i in root_vars:
    refugees_net_by_year[f'{i}_net'] = refugees_net_by_year[f'{i}_out'] - refugees_net_by_year[f'{i}_in']
    
#Save to CSV
refugees_net_by_year.to_csv('Processed Data/refugees_net_by_year.csv')

refugees_net_by_year.head()

,Refugees_in,Asylum_in,Returned refugees_in,Internally displaced persons_in,Returned IDPs_in,Stateless persons_in,Others of concern_in,Total Population_in,Refugees_out,Asylum_out,Returned refugees_out,Internally displaced persons_out,Returned IDPs_out,Stateless persons_out,Others of concern_out,Total Population_out,Refugees_net,Asylum_net,Returned refugees_net,Internally displaced persons_net,Returned IDPs_net,Stateless persons_net,Others of concern_net,Total Population_net
Year,,,,,,,,,,,,,,,,,,,,,,,,
1969,40000,0,0,0,0,0,0,40000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-40000.0,0.0,0.0,0.0,0.0,0.0,0.0,-40000.0
1970,40000,0,0,0,0,0,0,40000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-40000.0,0.0,0.0,0.0,0.0,0.0,0.0,-40000.0
1971,38000,0,0,0,0,0,0,38000,2000.0,0.0,0.0,0.0,0.0,0.0,0.0,2000.0,-36000.0,0.0,0.0,0.0,0.0,0.0,0.0,-36000.0
1972,37000,0,0,0,0,0,0,37000,2000.0,0.0,0.0,0.0,0.0,0.0,0.0,2000.0,-35000.0,0.0,0.0,0.0,0.0,0.0,0.0,-35000.0
1973,35200,0,0,0,0,0,0,35200,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-35200.0,0.0,0.0,0.0,0.0,0.0,0.0,-35200.0


In [7]:
refugees_in.head()

,Year,Country,Origin,Refugees,Asylum,Returned refugees,Internally displaced persons,Returned IDPs,Stateless persons,Others of concern,Total Population
0,1969,Brazil,Various/Unknown,40000,0,0,0,0,0,0,40000
1,1970,Brazil,Various/Unknown,40000,0,0,0,0,0,0,40000
2,1971,Brazil,Cuba,37800,0,0,0,0,0,0,37800
3,1971,Brazil,Various/Unknown,200,0,0,0,0,0,0,200
4,1972,Brazil,Cuba,200,0,0,0,0,0,0,200


In [8]:
#Make a dataset that summarizes the top 3 countries sending refugees to Brazil
refugees_in_top3_sources = refugees_in.loc[refugees_in.groupby('Year')['Total Population'].nlargest(3).index.get_level_values(1)]
#.drop(refugees_in.loc[refugees_in.groupby('Year')['Total Population'].nlargest(3).index.get_level_values(1)].columns[0], axis=1)
refugees_in_top3_sources.to_csv('Processed Data/refugees_in_top3_sources.csv')
refugees_in_top3_sources.head()

#Make the same dataset for the Top 5 countries sending refugees to Brazil
refugees_in_top5_sources = refugees_in.loc[refugees_in.groupby('Year')['Total Population'].nlargest(5).index.get_level_values(1)]
#.drop(refugees_in.loc[refugees_in.groupby('Year')['Total Population'].nlargest(5).index.get_level_values(1)].columns[0], axis=1)
refugees_in_top5_sources.to_csv('Processed Data/refugees_in_top5_sources.csv')
refugees_in_top5_sources.head()

,Year,Country,Origin,Refugees,Asylum,Returned refugees,Internally displaced persons,Returned IDPs,Stateless persons,Others of concern,Total Population
0,1969,Brazil,Various/Unknown,40000,0,0,0,0,0,0,40000
1,1970,Brazil,Various/Unknown,40000,0,0,0,0,0,0,40000
2,1971,Brazil,Cuba,37800,0,0,0,0,0,0,37800
3,1971,Brazil,Various/Unknown,200,0,0,0,0,0,0,200
5,1972,Brazil,Various/Unknown,36800,0,0,0,0,0,0,36800


In [9]:
#Summarize the movement for Top 3 by type, just to test what's going on
refugees_in_top3_sources[['Refugees','Asylum','Returned refugees','Internally displaced persons','Returned IDPs','Stateless persons','Others of concern','Total Population']].sum().reset_index().rename(columns={'index':'Category', '0': 'count'})

,Category,0
0,Refugees,556562
1,Asylum,35725
2,Returned refugees,268
3,Internally displaced persons,0
4,Returned IDPs,0
5,Stateless persons,0
6,Others of concern,97330
7,Total Population,689889


In [10]:
#Make a summary table for the Top 5 countries sending folks to Brazil
refugees_in_top5_sources[['Refugees','Asylum','Returned refugees','Internally displaced persons','Returned IDPs','Stateless persons','Others of concern','Total Population']].sum().reset_index().rename(columns={'index':'Category', '0': 'count'})

,Category,0
0,Refugees,571892
1,Asylum,44502
2,Returned refugees,475
3,Internally displaced persons,0
4,Returned IDPs,0
5,Stateless persons,0
6,Others of concern,97330
7,Total Population,714203


In [11]:
#Make a dataset that summarizes the top 3 countries Brazilian refugees go to
refugees_out_top3_sources = refugees_out.loc[refugees_out.groupby('Year')['Total Population'].nlargest(3).index.get_level_values(1)]
#.drop(refugees_out.loc[refugees_out.groupby('Year')['Total Population'].nlargest(3).index.get_level_values(1)].columns[0], axis=1)
refugees_out_top3_sources.to_csv('Processed Data/refugees_out_top3_sources.csv')
refugees_out_top3_sources.head()

#Make a dataset that summarizes the top 5 countries Brazilian refugees go to
refugees_out_top5_sources = refugees_out.loc[refugees_out.groupby('Year')['Total Population'].nlargest(5).index.get_level_values(1)]
#.drop(refugees_out.loc[refugees_out.groupby('Year')['Total Population'].nlargest(5).index.get_level_values(1)].columns[0], axis=1)
refugees_out_top5_sources.to_csv('Processed Data/refugees_out_top5_sources.csv')
refugees_out_top5_sources.head()

,Year,Country,Origin,Refugees,Asylum,Returned refugees,Internally displaced persons,Returned IDPs,Stateless persons,Others of concern,Total Population
0,1971,Chile,Brazil,2000,0,0,0,0,0,0,2000
1,1972,Chile,Brazil,2000,0,0,0,0,0,0,2000
2,1980,Italy,Brazil,5,0,0,0,0,0,0,5
3,1981,Italy,Brazil,5,0,0,0,0,0,0,5
4,1982,Italy,Brazil,5,0,0,0,0,0,0,5


In [12]:
#Summarize the movement for Top 3 by type, just to test what's going on
refugees_out_top3_sources[['Refugees','Asylum','Returned refugees','Internally displaced persons','Returned IDPs','Stateless persons','Others of concern','Total Population']].sum().reset_index().rename(columns={'index':'Category', '0': 'count'})

,Category,0
0,Refugees,17444
1,Asylum,14537
2,Returned refugees,0
3,Internally displaced persons,0
4,Returned IDPs,0
5,Stateless persons,91
6,Others of concern,0
7,Total Population,32072


In [13]:
#Summarize the movement for Top 5 by type, just to test what's going on
refugees_out_top5_sources[['Refugees','Asylum','Returned refugees','Internally displaced persons','Returned IDPs','Stateless persons','Others of concern','Total Population']].sum().reset_index().rename(columns={'index':'Category', '0': 'count'})

,Category,0
0,Refugees,17816
1,Asylum,14854
2,Returned refugees,9
3,Internally displaced persons,0
4,Returned IDPs,0
5,Stateless persons,197
6,Others of concern,0
7,Total Population,32876


In [14]:
#Make a count of how many times countries are found in the Top 3 or 5 source countries

refugees_in_top3_sources['Origin'].value_counts().reset_index().rename(columns={'index':'Country of Origin', 'Origin':'Years of Top 3 Status'}).to_csv('Processed Data/refugees_in_top3_sources_count.csv')
refugees_in_top3_sources['Origin'].value_counts().reset_index().rename(columns={'index':'Country of Origin', 'Origin':'Years of Top 3 Status'})

refugees_in_top5_sources['Origin'].value_counts().reset_index().rename(columns={'index':'Country of Origin', 'Origin':'Years of Top 5 Status'}).to_csv('Processed Data/refugees_in_top5_sources_count.csv')
refugees_in_top5_sources['Origin'].value_counts().reset_index().rename(columns={'index':'Country of Origin', 'Origin':'Years of Top 5 Status'})

,Country of Origin,Years of Top 5 Status
0,Various/Unknown,30
1,Angola,23
2,Dem. Rep. of the Congo,20
3,Liberia,20
4,Colombia,11
5,Cuba,9
6,Haiti,7
7,Sierra Leone,6
8,Romania,4
9,Senegal,4


In [15]:
#Make a count of how many times countries are found in the Top 3 or 5 destination countries

refugees_out_top3_sources['Country'].value_counts().reset_index().rename(columns={'index':'Destination Country', 'Country':'Years of Top 3 Status'}).to_csv('Processed Data/refugees_out_top3_sources_count.csv')
refugees_out_top3_sources['Country'].value_counts().reset_index().rename(columns={'index':'Destination Country', 'Country':'Years of Top 3 Status'})

refugees_out_top5_sources['Country'].value_counts().reset_index().rename(columns={'index':'Destination Country', 'Country':'Years of Top 5 Status'}).to_csv('Processed Data/refugees_out_top5_sources_count.csv')
refugees_out_top5_sources['Country'].value_counts().reset_index().rename(columns={'index':'Destination Country', 'Country':'Years of Top 5 Status'})

,Destination Country,Years of Top 5 Status
0,United States of America,24
1,Canada,23
2,Italy,15
3,Germany,15
4,Australia,14
5,Sweden,10
6,United Kingdom,6
7,Ireland,5
8,Belgium,3
9,Morocco,3


In [16]:
#Some preliminary tables that help us make the final timeline by country:

refugees_in_top5_sources_pivot = refugees_in_top5_sources[['Year','Origin']].copy()
refugees_in_top5_sources_pivot['Count'] = 1
refugees_in_top5_sources_pivot.head()

refugees_out_top5_sources_pivot = refugees_out_top5_sources[['Year','Country']].copy()
refugees_out_top5_sources_pivot['Count'] = 1
refugees_out_top5_sources_pivot.head()

,Year,Country,Count
0,1971,Chile,1
1,1972,Chile,1
2,1980,Italy,1
3,1981,Italy,1
4,1982,Italy,1


In [17]:
refugees_in_top5_by_year = refugees_in_top5_sources_pivot[['Year','Origin','Count']].pivot(index='Origin', columns='Year', values='Count')
refugees_in_top5_by_year.fillna('', inplace=True)
refugees_in_top5_by_year.replace([1], ['*'], inplace=True) 
refugees_in_top5_by_year.to_csv('Processed Data/refugees_in_top5_by_year.csv')
refugees_in_top5_by_year

Year,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
Origin,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Angola,,,,,,,,,,,,,,,,,,,,,,,,,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,,*,*
Bangladesh,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,*,*,,
Chile,,,,,,,,,,,,,,,,,,*,,,,,,,*,*,*,,,,,,,,,,,,,,,,,,,,,
Colombia,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,*,*,*,*,*,*,*,*,*,*,*,,
Cuba,,,*,*,,,,,,,,,,,,,,,,,,,,,,,,*,*,*,*,,,,,,*,*,*,,,,,,,,,
Dem. Rep. of the Congo,,,,,,,,,,,,,,,,,,,,,,,,,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,,,,
Haiti,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,*,*,*,*,*,*,*
Iraq,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,*,*,,,,,,,
Liberia,,,,,,,,,,,,,,,,,,,,,,,,,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,,,,


In [18]:
refugees_out_top5_by_year = refugees_out_top5_sources_pivot[['Year','Country','Count']].pivot(index='Country', columns='Year', values='Count')
refugees_out_top5_by_year.fillna('', inplace=True)
refugees_out_top5_by_year.replace([1], ['*'], inplace=True) 
refugees_out_top5_by_year.to_csv('Processed Data/refugees_out_top5_by_year.csv')
refugees_out_top5_by_year

Year,1971,1972,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
Country,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Australia,,,,,,,,,,,,,,,,,,,*,*,*,*,*,*,*,*,*,*,*,*,*,,*,,,,,,
Belgium,,,,,,,,,,,,,,,,,,,,,*,*,*,,,,,,,,,,,,,,,,
Brazil,,,,,,,,,,,,,,,,,,,,,,,,,,,,*,,,,*,,,,,,,
Canada,,,,,,,,,,,,,,,,,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*
Chile,*,*,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Cuba,,,,,,,,,,,,,,,,,*,,,,,,,,,,,,,,,,,,,,,,
Denmark,,,,,,,,,,,,*,*,,,,,,,,,,,,,,,,,,,,,,,,,,
France,,,,,,,,,,,,,,,,,,,,,,,,,,,,,*,,,,,,,,,,
Germany,,,,,,,,,,,,,,,,,,,,,,,,,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*


In [19]:
#Create dataframe with just the years in it and the order for merging

years_df = pd.DataFrame(np.repeat(list(range(1969,2017)),5), columns=['Year'])

from itertools import cycle

seq = cycle([1, 2, 3, 4, 5])
years_df['Order'] = [next(seq) for count in range(years_df.shape[0])]

years_df[:10]

,Year,Order
0,1969,1
1,1969,2
2,1969,3
3,1969,4
4,1969,5
5,1970,1
6,1970,2
7,1970,3
8,1970,4
9,1970,5


In [20]:
#Prepare the top 5 refugees out database for merging.

out_for_merge= refugees_out_top5_sources[['Year','Country','Total Population']].copy()
out_for_merge["Order"] = out_for_merge.groupby("Year").cumcount()+1
out_for_merge.head()

in_for_merge= refugees_in_top5_sources[['Year','Origin','Total Population']].copy()
in_for_merge["Order"] = in_for_merge.groupby("Year").cumcount()+1
in_for_merge.head()

,Year,Origin,Total Population,Order
0,1969,Various/Unknown,40000,1
1,1970,Various/Unknown,40000,1
2,1971,Cuba,37800,1
3,1971,Various/Unknown,200,2
5,1972,Various/Unknown,36800,1


In [21]:
#Merge the two dataframes together to prepare for merging

out_for_export = pd.merge(years_df, out_for_merge, left_on=["Year","Order"], right_on=["Year","Order"], how='left')

out_for_export.fillna(0,inplace=True)
out_for_export.head()

in_for_export = pd.merge(years_df, in_for_merge, left_on=["Year","Order"], right_on=["Year","Order"], how='left')

in_for_export.fillna(0,inplace=True)
in_for_export.head()

,Year,Order,Origin,Total Population
0,1969,1,Various/Unknown,40000.0
1,1969,2,0,0.0
2,1969,3,0,0.0
3,1969,4,0,0.0
4,1969,5,0,0.0


In [22]:
in_for_export['New Pop'] = in_for_export['Total Population']/10
in_for_export[in_for_export['New Pop']>0]['New Pop'].describe(percentiles=[.1,.2,.3,.4,.5,.6,.7,.8,.9,1])

count     151.000000
mean      472.982119
std       978.966270
min         2.700000
10%        14.000000
20%        16.400000
30%        22.400000
40%        26.000000
50%        60.600000
60%       136.800000
70%       203.300000
80%       521.900000
90%      1757.900000
100%     4000.000000
max      4000.000000
Name: New Pop, dtype: float64

In [23]:
out_for_export['New Pop'] = out_for_export['Total Population']/10
out_for_export[out_for_export['New Pop']>0]['New Pop'].describe(percentiles=[.1,.2,.3,.4,.5,.6,.7,.8,.9,1])

count    134.000000
mean      24.534328
std       53.929768
min        0.100000
10%        0.400000
20%        0.700000
30%        1.090000
40%        1.720000
50%        2.750000
60%        6.800000
70%       15.400000
80%       26.220000
90%       74.670000
100%     407.200000
max      407.200000
Name: New Pop, dtype: float64

In [24]:
in_for_export_type1 = in_for_export.copy()
in_for_export_type2 = in_for_export.copy()
in_for_export_type3 = in_for_export.copy()
in_for_export_type4 = in_for_export.copy()

out_for_export_type1 = out_for_export.copy()
out_for_export_type2 = out_for_export.copy()
out_for_export_type3 = out_for_export.copy()
out_for_export_type4 = out_for_export.copy()

In [25]:
#in_for_export_type1['New Pop'].round(decimals = 0) 
#in_for_export_type1['New Pop'] = in_for_export_type1['New Pop'].astype(int)
a = np.array(in_for_export_type1['New Pop'].values.tolist())
b = np.array(out_for_export_type1['New Pop'].values.tolist())
in_for_export_type2['New Pop'] = np.where(a > 100, 100, a).tolist()
in_for_export_type3['New Pop'] = np.where(a > 100, 200, a).tolist()
in_for_export_type4['New Pop'] = np.where(a > 100, 500, a).tolist()

out_for_export_type2['New Pop'] = np.where(b > 100, 100, b).tolist()
out_for_export_type3['New Pop'] = np.where(b > 100, 200, b).tolist()
out_for_export_type4['New Pop'] = np.where(b > 100, 500, b).tolist()

In [26]:
#Create a list based on the total population that allows us to make a histogram

out_pop_count = [int(i) for i in out_for_export['Total Population'].values.tolist()]
in_pop_count = [int(i) for i in in_for_export['Total Population'].values.tolist()]
out_pop_count_type1 = [int(i) for i in out_for_export_type1['New Pop'].values.tolist()]
in_pop_count_type1 = [int(i) for i in in_for_export_type1['New Pop'].values.tolist()]
out_pop_count_type2 = [int(i) for i in out_for_export_type2['New Pop'].values.tolist()]
in_pop_count_type2 = [int(i) for i in in_for_export_type2['New Pop'].values.tolist()]
out_pop_count_type3 = [int(i) for i in out_for_export_type3['New Pop'].values.tolist()]
in_pop_count_type3 = [int(i) for i in in_for_export_type3['New Pop'].values.tolist()]
out_pop_count_type4 = [int(i) for i in out_for_export_type4['New Pop'].values.tolist()]
in_pop_count_type4 = [int(i) for i in in_for_export_type4['New Pop'].values.tolist()]

def repeat_to_length(string_to_expand, length):
    cell = (string_to_expand * (int(length/len(string_to_expand))+1))[:length]
    graph_list.append(cell)

In [27]:
#Add the histogram column to the initial refugees out dataframe
graph_list = []

for i in out_pop_count:
    repeat_to_length('|', i)
    
pd.set_option('display.max_colwidth', -1)
grapharray = np.asarray(graph_list)
out_for_export['Histogram'] = grapharray
out_for_export

#Add the histogram column to the initial refugees in dataframe

graph_list = []

for i in in_pop_count:
    repeat_to_length('|', i)
    
pd.set_option('display.max_colwidth', -1)
grapharray = np.asarray(graph_list)
in_for_export['Histogram'] = grapharray
in_for_export

Year  Order                              Origin  Total Population  \
0    1969  1      Various/Unknown                     40000.0            
1    1969  2      0                                   0.0                
2    1969  3      0                                   0.0                
3    1969  4      0                                   0.0                
4    1969  5      0                                   0.0                
5    1970  1      Various/Unknown                     40000.0            
6    1970  2      0                                   0.0                
7    1970  3      0                                   0.0                
8    1970  4      0                                   0.0                
9    1970  5      0                                   0.0                
10   1971  1      Cuba                                37800.0            
11   1971  2      Various/Unknown                     200.0              
12   1971  3      0                                   0.0                
13   1971  4      0                                   0.0                
14   1971  5      0                                   0.0                
15   1972  1      Various/Unknown                     36800.0            
16   1972  2      Cuba                                200.0              
17   1972  3      0                                   0.0                
18   1972  4      0                                   0.0                
19   1972  5      0                                   0.0                
20   1973  1      Various/Unknown                     35200.0            
21   1973  2      0                                   0.0                
22   1973  3      0                                   0.0                
23   1973  4      0                                   0.0                
24   1973  5      0                                   0.0                
25   1974  1      Various/Unknown                     34000.0            
26   1974  2      0                                   0.0                
27   1974  3      0                                   0.0                
28   1974  4      0                                   0.0                
29   1974  5      0                                   0.0                
..    ... ..     ..                                   ...                
210  2011  1      Haiti                               3875.0             
211  2011  2      Angola                              1709.0             
212  2011  3      Colombia                            764.0              
213  2011  4      Dem. Rep. of the Congo              505.0              
214  2011  5      Liberia                             259.0              
215  2012  1      Haiti                               5583.0             
216  2012  2      Angola                              1711.0             
217  2012  3      Colombia                            985.0              
218  2012  4      Dem. Rep. of the Congo              606.0              
219  2012  5      Liberia                             248.0              
220  2013  1      Haiti                               12321.0            
221  2013  2      Bangladesh                          1711.0             
222  2013  3      Colombia                            1270.0             
223  2013  4      Angola                              1104.0             
224  2013  5      Senegal                             826.0              
225  2014  1      Haiti                               29241.0            
226  2014  2      Senegal                             2254.0             
227  2014  3      Bangladesh                          1951.0             
228  2014  4      Syrian Arab Rep.                    1815.0             
229  2014  5      Colombia                            1371.0             
230  2015  1      Haiti                               6271.0             
231  2015  2      Senegal                             5219.0             
232  2015

In [28]:
#TYPE 1 - Reduced by a factor of 10

#Add the histogram column to the initial refugees out dataframe
graph_list = []

for i in out_pop_count_type1:
    repeat_to_length('|', i)
    
pd.set_option('display.max_colwidth', -1)
grapharray = np.asarray(graph_list)
out_for_export_type1['Histogram'] = grapharray
out_for_export_type1.head()

#Add the histogram column to the initial refugees in dataframe

graph_list = []

for i in in_pop_count_type1:
    repeat_to_length('|', i)
    
pd.set_option('display.max_colwidth', -1)
grapharray = np.asarray(graph_list)
in_for_export_type1['Histogram'] = grapharray
in_for_export_type1.head()

#TYPE 2 - Reduced by a factor of 10 and 100 CUT OFF

#Add the histogram column to the initial refugees out dataframe
graph_list = []

for i in out_pop_count_type2:
    repeat_to_length('|', i)
    
pd.set_option('display.max_colwidth', -1)
grapharray = np.asarray(graph_list)
out_for_export_type2['Histogram'] = grapharray
out_for_export_type2.head()

#Add the histogram column to the initial refugees in dataframe

graph_list = []

for i in in_pop_count_type2:
    repeat_to_length('|', i)
    
pd.set_option('display.max_colwidth', -1)
grapharray = np.asarray(graph_list)
in_for_export_type2['Histogram'] = grapharray
in_for_export_type2.head()

#TYPE 3 - Reduced by a factor of 10 and 200 CUT OFF

#Add the histogram column to the initial refugees out dataframe
graph_list = []

for i in out_pop_count_type3:
    repeat_to_length('|', i)
    
pd.set_option('display.max_colwidth', -1)
grapharray = np.asarray(graph_list)
out_for_export_type3['Histogram'] = grapharray
out_for_export_type3.head()

#Add the histogram column to the initial refugees in dataframe

graph_list = []

for i in in_pop_count_type3:
    repeat_to_length('|', i)
    
pd.set_option('display.max_colwidth', -1)
grapharray = np.asarray(graph_list)
in_for_export_type3['Histogram'] = grapharray
in_for_export_type3.head()

#TYPE 4 - Reduced by a factor of 10 and 500 CUT OFF

#Add the histogram column to the initial refugees out dataframe
graph_list = []

for i in out_pop_count_type4:
    repeat_to_length('|', i)
    
pd.set_option('display.max_colwidth', -1)
grapharray = np.asarray(graph_list)
out_for_export_type4['Histogram'] = grapharray
out_for_export_type4.head()

#Add the histogram column to the initial refugees in dataframe

graph_list = []

for i in in_pop_count_type4:
    repeat_to_length('|', i)
    
pd.set_option('display.max_colwidth', -1)
grapharray = np.asarray(graph_list)
in_for_export_type4['Histogram'] = grapharray
in_for_export_type4.head()

,Year,Order,Origin,Total Population,New Pop,Histogram
0,1969,1,Various/Unknown,40000.0,500.0,||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
1,1969,2,0,0.0,0.0,
2,1969,3,0,0.0,0.0,
3,1969,4,0,0.0,0.0,
4,1969,5,0,0.0,0.0,


In [32]:
#Clean up that export dataframe for refugees out
out_for_export['Total Population'].replace([0.0],[''],inplace=True)
out_for_export['Country'].replace([0],[''],inplace=True)
out_for_export['New Pop'].replace([0],[''],inplace=True)
out_for_export.drop('Order', axis=1).to_csv('Processed Data/refugees_out_final.csv')
out_for_export.drop(['Order','Total Population','New Pop'], axis=1).to_csv('Processed Data/refugees_out_final_for_print.csv')

out_for_export_type1['Total Population'].replace([0.0],[''],inplace=True)
out_for_export_type1['Country'].replace([0],[''],inplace=True)
out_for_export_type1['New Pop'].replace([0],[''],inplace=True)
out_for_export_type1.drop('Order', axis=1).to_csv('Processed Data/refugees_out_final_type1.csv')
out_for_export_type1.drop(['Order','Total Population','New Pop'], axis=1).to_csv('Processed Data/refugees_out_final_for_print_type1.csv')

out_for_export_type2['Total Population'].replace([0.0],[''],inplace=True)
out_for_export_type2['Country'].replace([0],[''],inplace=True)
out_for_export_type2['New Pop'].replace([0],[''],inplace=True)
out_for_export_type2.drop('Order', axis=1).to_csv('Processed Data/refugees_out_final_type2.csv')
out_for_export_type2.drop(['Order','Total Population','New Pop'], axis=1).to_csv('Processed Data/refugees_out_final_for_print_type2.csv')

out_for_export_type3['Total Population'].replace([0.0],[''],inplace=True)
out_for_export_type3['Country'].replace([0],[''],inplace=True)
out_for_export_type3['New Pop'].replace([0],[''],inplace=True)
out_for_export_type3.drop('Order', axis=1).to_csv('Processed Data/refugees_out_final_type3.csv')
out_for_export_type3.drop(['Order','Total Population','New Pop'], axis=1).to_csv('Processed Data/refugees_out_final_for_print_type3.csv')

out_for_export_type4['Total Population'].replace([0.0],[''],inplace=True)
out_for_export_type4['Country'].replace([0],[''],inplace=True)
out_for_export_type4['New Pop'].replace([0],[''],inplace=True)
out_for_export_type4.drop('Order', axis=1).to_csv('Processed Data/refugees_out_final_type4.csv')
out_for_export_type4.drop(['Order','Total Population','New Pop'], axis=1).to_csv('Processed Data/refugees_out_final_for_print_type4.csv')


In [33]:
#Clean up that export dataframe for refugees in
in_for_export['Total Population'].replace([0.0],[''],inplace=True)
in_for_export['Origin'].replace([0],[''],inplace=True)
in_for_export['New Pop'].replace([0],[''],inplace=True)
in_for_export.drop('Order', axis=1).to_csv('Processed Data/refugees_in_final.csv')
in_for_export.drop(['Order','Total Population','New Pop'], axis=1).to_csv('Processed Data/refugees_in_final_for_print.csv')

in_for_export_type1['Total Population'].replace([0.0],[''],inplace=True)
in_for_export_type1['Origin'].replace([0],[''],inplace=True)
in_for_export_type1['New Pop'].replace([0],[''],inplace=True)
in_for_export_type1.drop('Order', axis=1).to_csv('Processed Data/refugees_in_final_type1.csv')
in_for_export_type1.drop(['Order','Total Population','New Pop'], axis=1).to_csv('Processed Data/refugees_in_final_for_print_type1.csv')

in_for_export_type2['Total Population'].replace([0.0],[''],inplace=True)
in_for_export_type2['Origin'].replace([0],[''],inplace=True)
in_for_export_type2['New Pop'].replace([0],[''],inplace=True)
in_for_export_type2.drop('Order', axis=1).to_csv('Processed Data/refugees_in_final_type2.csv')
in_for_export_type2.drop(['Order','Total Population','New Pop'], axis=1).to_csv('Processed Data/refugees_in_final_for_print_type2.csv')

in_for_export_type3['Total Population'].replace([0.0],[''],inplace=True)
in_for_export_type3['Origin'].replace([0],[''],inplace=True)
in_for_export_type3['New Pop'].replace([0],[''],inplace=True)
in_for_export_type3.drop('Order', axis=1).to_csv('Processed Data/refugees_in_final_type3.csv')
in_for_export_type3.drop(['Order','Total Population','New Pop'], axis=1).to_csv('Processed Data/refugees_in_final_for_print_type3.csv')

in_for_export_type4['Total Population'].replace([0.0],[''],inplace=True)
in_for_export_type4['Origin'].replace([0],[''],inplace=True)
in_for_export_type4['New Pop'].replace([0],[''],inplace=True)
in_for_export_type4.drop('Order', axis=1).to_csv('Processed Data/refugees_in_final_type4.csv')
in_for_export_type4.drop(['Order','Total Population','New Pop'], axis=1).to_csv('Processed Data/refugees_in_final_for_print_type4.csv')
